# Bước 0:

In [5]:
import pandas as pd
url = "https://raw.githubusercontent.com/huongd17at089/income_prediction/master/data_adult.csv"
df = pd.read_csv(url)

In [6]:
df.sample(5)

,Unnamed: 0,X,age,workclass,education,educational.num,marital.status,race,gender,hours.per.week,income
44910,44911,44911,28,Private,Bachelors,13,Never-married,Black,Female,45,<=50K
43968,43969,43969,62,Private,Assoc-voc,11,Married-civ-spouse,White,Male,40,>50K
27501,27502,27502,35,Private,Some-college,10,Married-civ-spouse,White,Male,55,>50K
16902,16903,16903,51,Private,HS-grad,9,Married-civ-spouse,Black,Male,40,>50K
32857,32858,32858,23,Private,Some-college,10,Never-married,White,Female,50,<=50K
